<a href="https://colab.research.google.com/github/MariaZharova/Skillfactory_projects/blob/main/MFTIDS/NLP_lectures/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_%D0%BB%D0%B5%D0%BA%D1%86%D0%B8%D1%8F_6_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets --use-deprecated=legacy-resolver > /dev/null

ERROR: pip's legacy dependency resolver does not consider dependency conflicts when selecting packages. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you'll have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you'll have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip install pyarrow==15.0.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 49.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 17.0.0
    Uninstalling pyarrow-17.0.0:
      Successfully uninstalled pyarrow-17.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 15.0.2 which is incompatible.


In [ ]:
import time
import numpy as np
import pandas as pd
import transformers
import torch
import torch.nn.functional as F
from datasets import load_dataset
from torch import cuda
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.dataset import random_split
from transformers import DistilBertModel, DistilBertTokenizer
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [ ]:
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
raw_datasets = load_dataset("imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
class AppendRandomLetters:
  def __init__(self, charset, append_prob):
    self.charset = charset
    self.append_prob = append_prob

  def _append_letter(self, data):
    charset = self.charset if self.charset else list(data + string.punctuation)

    letters = []
    for letter in data:
      letters.append(letter)
      if random.random() < self.append_prob:
        random_letter = random.choice(charset)
        letters.append(random_letter)
      return ''.join(letters)

# Part 1.
Получаем эмбеддинги из предобученной модели и решаем задачу классификаиции на них при помощи классических моделей.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")
model.to(device)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li

In [ ]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [ ]:
print(embed_bert_cls('hello world', model, tokenizer).shape)

(768,)


In [ ]:
x = np.zeros((len(raw_datasets['train']), 768))
y = np.array(raw_datasets['train']['label'])

In [ ]:
for ind, sent in enumerate(tqdm(raw_datasets['train']['text'])):
  x[ind] = embed_bert_cls(sent, model, tokenizer)

100%|██████████| 25000/25000 [04:43<00:00, 88.32it/s] 


In [ ]:
x_test = np.zeros((len(raw_datasets['test']), 768))
y_test = np.array(raw_datasets['test']['label'])

In [ ]:
for ind, sent in enumerate(tqdm(raw_datasets['test']['text'])):
  x_test[ind] = embed_bert_cls(sent, model, tokenizer)

100%|██████████| 25000/25000 [04:41<00:00, 88.85it/s]


In [ ]:
logit = LogisticRegression(random_state=2029, solver='sag')
logit.fit(x, y)

LogisticRegression(random_state=2029, solver='sag')

In [ ]:
y_preds = logit.predict(x_test)

In [ ]:
accuracy_score(y_test, y_preds)

0.85568

In [ ]:
parameters = {'C': np.linspace(0.0001, 100, num=50)}

clf = GridSearchCV(logit, parameters)
clf.fit(x, y)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which 

GridSearchCV(estimator=LogisticRegression(random_state=2029, solver='sag'),
             param_grid={'C': array([1.00000000e-04, 2.04091429e+00, 4.08172857e+00, 6.12254286e+00,
       8.16335714e+00, 1.02041714e+01, 1.22449857e+01, 1.42858000e+01,
       1.63266143e+01, 1.83674286e+01, 2.04082429e+01, 2.24490571e+01,
       2.44898714e+01, 2.65306857e+01, 2.85715000e+01, 3.06123143e+01,
       3.26531286e+0...
       4.89796429e+01, 5.10204571e+01, 5.30612714e+01, 5.51020857e+01,
       5.71429000e+01, 5.91837143e+01, 6.12245286e+01, 6.32653429e+01,
       6.53061571e+01, 6.73469714e+01, 6.93877857e+01, 7.14286000e+01,
       7.34694143e+01, 7.55102286e+01, 7.75510429e+01, 7.95918571e+01,
       8.16326714e+01, 8.36734857e+01, 8.57143000e+01, 8.77551143e+01,
       8.97959286e+01, 9.18367429e+01, 9.38775571e+01, 9.59183714e+01,
       9.79591857e+01, 1.00000000e+02])})

In [ ]:
clf.best_params_

{'C': 65.30615714285715}

In [ ]:
logit = LogisticRegression(random_state=2029, solver='sag', **clf.best_params_)
logit.fit(x, y)
y_preds = logit.predict(x_test)
accuracy_score(y_test, y_preds)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.87548

# Part 2.
Добавляем в конец предобученной модели линейный слой-классификатор. Важно заморозить слои предобученной модели.

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
class CustomDataset:
  def __init__(self, data, targets, tokenizer, max_len=512):
    self.data = data
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
        text = str(self.data[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.long)
        }

In [ ]:
train_dataset = CustomDataset(raw_datasets['train']['text'],
                              raw_datasets['train']['label'],
                              tokenizer)

In [ ]:
def train(dataloader):
    model.train()
    total_acc, total_count, total_loss = 0, 0, 0
    start_time = time.time()

    for idx, data in enumerate(tqdm(dataloader)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        optimizer.zero_grad()
        outputs = model(ids, mask, token_type_ids)
        loss = criterion(outputs, targets)
        total_loss += loss
        loss.backward()
        optimizer.step()
        outputs = outputs.detach().cpu().numpy()
        targets = targets.to('cpu').numpy()
        total_acc += accuracy(outputs, targets)
        total_count += len(targets)
    elapsed = time.time() - start_time
    print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f} | loss {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count, total_loss/total_count))
    total_acc, total_count = 0, 0
    start_time = time.time()

In [ ]:
def evaluate(dataloader):
    model.eval()
    total_acc, total_count, total_loss = 0, 0, 0

    with torch.no_grad():
        for idx, data in enumerate(tqdm(dataloader)):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            label = data['targets'].to(device, dtype=torch.long)

            predited_label = model(ids, mask, token_type_ids)
            loss = criterion(predited_label, label)
            total_loss += loss
            predited_label = predited_label.detach().cpu().numpy()
            label = label.to('cpu').numpy()
            total_acc += accuracy(predited_label, label)
            total_count += len(label)
    return total_acc/total_count, total_loss/total_count

In [ ]:
def accuracy(probs, targets):
    outputs = np.argmax(probs, axis=1)
    return np.sum(outputs == targets)

In [ ]:
# гиперпараметры
EPOCHS = 4
LR = 3e-4
BATCH_SIZE = 16

model = DistilBERTClass().to(device)
model.l1.requires_grad_(False)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-4)
total_accu = None

num_train = int(len(train_dataset) * 0.8)

split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = torch.utils.data.DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val, loss_val = evaluate(valid_dataloader)
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} | valid loss {:8.3f}'.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val, loss_val))
    print('-' * 59)

  0%|          | 0/1250 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 1250/1250 [07:13<00:00,  2.89it/s]


| epoch   1 |  1249/ 1250 batches | accuracy    0.822 | loss    0.024


100%|██████████| 313/313 [01:44<00:00,  3.00it/s]


-----------------------------------------------------------
| end of epoch   1 | time: 537.23s | valid accuracy    0.868 | valid loss    0.020
-----------------------------------------------------------


100%|██████████| 1250/1250 [07:11<00:00,  2.90it/s]


...

-----------------------------------------------------------
| end of epoch   3 | time: 535.58s | valid accuracy    0.874 | valid loss    0.020
-----------------------------------------------------------


100%|██████████| 1250/1250 [07:11<00:00,  2.90it/s]


| epoch   4 |  1249/ 1250 batches | accuracy    0.845 | loss    0.022


100%|██████████| 313/313 [01:43<00:00,  3.02it/s]

-----------------------------------------------------------
| end of epoch   4 | time: 535.18s | valid accuracy    0.869 | valid loss    0.019
-----------------------------------------------------------


In [ ]:
for epoch in range(4,5):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val, loss_val = evaluate(valid_dataloader)
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} | valid loss {:8.3f}'.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val, loss_val))
    print('-' * 59)

100%|██████████| 1250/1250 [07:10<00:00,  2.90it/s]


| epoch   4 |  1249/ 1250 batches | accuracy    0.850 | loss    0.022


100%|██████████| 313/313 [01:44<00:00,  3.00it/s]

-----------------------------------------------------------
| end of epoch   4 | time: 535.35s | valid accuracy    0.864 | valid loss    0.020
-----------------------------------------------------------
